In [ ]:
import mqr
from mqr.plot import Figure
from mqr.nbtools import hstack, vstack

In [ ]:
import numpy as np
import pandas as pd

# Datasets

In [ ]:
# Silicon wafer resistivity - NIST 
# F 84 - 99

columns = ['RUNID', 'WAFERID', 'PROBE', 'MONTH', 'DAY', 'OPERATOR', 'TEMP', 'AVERAGE', 'STDDEV',]
dtype = {
    'WAFERID': int,
    'PROBE': int,
}

data = pd.read_csv(
    'https://www.itl.nist.gov/div898/software/dataplot/data/MPC61.DAT',
    skiprows=50,
    header=None,
    names=columns,
    sep='\\s+',
    dtype=dtype,
    storage_options={'user-agent': 'github:nklsxn/mqr'},
)
data.head()

---
# GRR

In [ ]:
# Use tolerance of 8 Ohm cm, and test for variance contribution from the probe (listed as operator)
tol = 2*8.0
names = mqr.msa.NameMapping(part='WAFERID', operator='PROBE', measurement='AVERAGE')
names

In [ ]:
# Use only RUNID==1, as in the lecture notes
grr = mqr.msa.GRR(
    data.query('RUNID==1'),
    tolerance=tol,
    names=names,
    include_interaction=True)

vstack(
    grr, # Default display in a notebook is the same as displaying `mqr.msa.SummaryTable(grr)`
    mqr.anova.summary(grr.regression_result),
    align_items='center',
)

In [ ]:
df = mqr.anova.adequacy(grr.regression_result)

df

NB: one outlier skews the residuals. It is visible at the very low quantiles in the probability plot.
* The outlier is low enough to reject the JB null-hyp with significance >95%.
* The Anderson-Darling test doesn't find enough evidence to reject the null-hyp that the residuals are normal.

Consider reviewing the test conditions for that datapoint to ensure the data is likely from the same process as the others.

In [ ]:
reg_result = grr.regression_result

with Figure(6, 4, 2, 2) as (fig, ax):
    mqr.plot.regression.residuals(reg_result.resid, reg_result.fittedvalues, axs=ax)
    plot = mqr.nbtools.grab_figure(fig)

vstack(
    mqr.anova.adequacy(reg_result),
    mqr.anova.summary(reg_result),
    hstack(
        plot,
        mqr.inference.dist.test_1sample(reg_result.resid),
    ),
)

---
# Models

In [ ]:
mqr.anova.groups(
    reg_result,
    value=names.measurement,
    factor=names.operator)

In [ ]:
mqr.anova.interactions(
    grr.data,
    value=names.measurement,
    between=(names.part, names.operator))

---
## Plots

In [ ]:
table = mqr.msa.VarianceTable(grr)
table

In [ ]:
with Figure(10, 6, 3, 2) as (fig, ax):
    mqr.plot.msa.grr(grr, axs=ax)